In [41]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
#from mlfs import config_Sensor_1
#settings = config_Sensor_1.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/karen/mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [42]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config_Sensor_1
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [43]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR1_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-16 19:12:50,173 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-16 19:12:50,175 INFO: Initializing external client
2025-11-16 19:12:50,175 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 19:12:51,808 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279151


'{"country": "italy", "city": "rome", "street": "ViaClelia", "aqicn_url": "https://api.waqi.info/feed/A98194/", "latitude": 41.89, "longitude": 12.48}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [44]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_lagged_sensor_1',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_sensor',
    version=1,
)

In [47]:
a = air_quality_fg.read()
a

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.99s) 


,date,pm25,lag_1,lag_2,lag_3,country,city,street,url
0,2020-09-05 00:00:00+00:00,3.15,2.50,2.75,2.17,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
1,2020-07-27 00:00:00+00:00,3.12,2.00,2.87,3.35,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
2,2020-04-15 00:00:00+00:00,2.85,13.63,15.40,14.53,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
3,2025-10-21 00:00:00+00:00,6.12,9.00,8.26,6.70,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
4,2025-10-24 00:00:00+00:00,4.70,5.27,7.19,6.12,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
...,...,...,...,...,...,...,...,...,...
2075,2021-06-04 00:00:00+00:00,3.38,4.12,3.53,3.15,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
2076,2020-12-06 00:00:00+00:00,2.29,3.17,4.65,8.68,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
2077,2025-05-19 00:00:00+00:00,3.90,2.22,1.60,1.60,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/
2078,2024-12-15 00:00:00+00:00,4.28,10.55,6.30,7.78,italy,rome,ViaClelia,https://api.waqi.info/feed/A98194/


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [45]:
import pandas as pd
from datetime import timedelta

# Get today's air quality data
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)

# Load full feature group as pandas DataFrame
air_quality_df = air_quality_fg.read()

# Convert 'today' and lags to date objects (drop any time component)
today_date = pd.to_datetime(today).date()
yesterday = today_date - timedelta(days=1)
before_yesterday = today_date - timedelta(days=2)
before_2_yesterday = today_date - timedelta(days=3)

# Helper: compare only on date part (no new columns)
def match_by_date(df, target_date):
    mask = df['date'].dt.date == target_date
    return df.loc[mask, 'pm25']

# Extract lag values
lag_1_series = match_by_date(air_quality_df, yesterday)
lag_2_series = match_by_date(air_quality_df, before_yesterday)
lag_3_series = match_by_date(air_quality_df, before_2_yesterday)

# Get scalar values (default to NaN if missing)
lag_1 = lag_1_series.iloc[0] if not lag_1_series.empty else float('nan')
lag_2 = lag_2_series.iloc[0] if not lag_2_series.empty else float('nan')
lag_3 = lag_3_series.iloc[0] if not lag_3_series.empty else float('nan')

# Add lag values as new columns to today's DF
aq_today_df['lag_1'] = lag_1
aq_today_df['lag_2'] = lag_2
aq_today_df['lag_3'] = lag_3

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.00s) 


,pm25,country,city,street,date,url,lag_1,lag_2,lag_3
0,27.0,italy,rome,ViaClelia,2025-11-16,https://api.waqi.info/feed/A98194/,NaN,9.15,9.75


In [39]:
# PM2.5 for today
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
 6   lag_1    0 non-null      float64       
 7   lag_2    1 non-null      float32       
 8   lag_3    1 non-null      float32       
dtypes: datetime64[ns](1), float32(3), float64(1), object(4)
memory usage: 188.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [40]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

,date,temperature_2m_mean,apparent_temperature_mean,dewpoint_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,city
0,2025-11-16,21.200001,19.171083,10.662126,0.0,0.0,0.0,15.328561,170.537750,388.0,0.282626,rome
1,2025-11-17,19.200001,17.112673,12.694678,0.1,0.1,0.0,20.124611,222.825241,370.0,0.234400,rome
2,2025-11-18,16.299999,16.742821,12.260189,0.3,0.3,0.0,1.800000,360.000000,274.0,0.154788,rome
3,2025-11-19,12.950000,11.919285,9.766445,1.2,1.2,0.0,6.725354,344.475830,225.0,0.118642,rome
4,2025-11-20,13.450000,11.935459,12.175789,0.7,0.7,0.0,15.038350,168.959106,118.0,0.065327,rome
5,2025-11-21,9.300000,7.006087,8.539822,2.2,2.2,0.0,13.104198,344.054535,93.0,0.045752,rome
6,2025-11-22,15.600000,12.885321,12.161602,2.1,2.1,0.0,23.233252,130.601212,201.0,0.122916,rome


In [36]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   apparent_temperature_mean    7 non-null      float32       
 3   dewpoint_2m_mean             7 non-null      float32       
 4   precipitation_sum            7 non-null      float32       
 5   rain_sum                     7 non-null      float32       
 6   snowfall_sum                 7 non-null      float32       
 7   wind_speed_10m_max           7 non-null      float32       
 8   wind_direction_10m_dominant  7 non-null      float32       
 9   shortwave_radiation_sum      7 non-null      float32       
 10  et0_fao_evapotranspiration   7 non-null      float32       
 11  city                         7 non-null      obje

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [31]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-16 19:09:31,777 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279151/fs/1265760/fg/1718676


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time:


Launching job: air_quality_lagged_sensor_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279151/jobs/named/air_quality_lagged_sensor_3_1_offline_fg_materialization/executions


(Job('air_quality_lagged_sensor_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758844
         }
       },
       "result": {
         "observed_value": 25.999998092651367,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T06:09:31.000776Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations"

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-16 18:47:52,626 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279151/fs/1265760/fg/1718623


Uploading Dataframe: 100.00% |█| Rows 7/7 | Elapsed Time:


Launching job: weather_sensor_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279151/jobs/named/weather_sensor_1_offline_fg_materialization/executions
2025-11-16 18:48:10,664 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-16 18:48:13,906 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-16 18:50:00,736 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-16 18:50:00,916 INFO: Waiting for log aggregation to finish.
2025-11-16 18:50:09,601 INFO: Execution finished successfully.


(Job('weather_sensor_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758801
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T05:47:52.000625Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
